In [1]:
#The goal is to write a script that given a movie name returns similar movies.

#Also, truncate the dataset to top 1000 movies in terms of review count. This should give more robust results.

In [2]:
import numpy as np
import pandas as pd

ratings = pd.read_csv('https://raw.githubusercontent.com/jputrius/ml_intro/refs/heads/main/data/movielens/ratings.csv')
movies = pd.read_csv('http://raw.githubusercontent.com/jputrius/ml_intro/refs/heads/main/data/movielens/movies.csv')

In [3]:
topid = ratings.groupby("movieId").size().sort_values(ascending=False)[0:1000].index
top = movies.set_index("movieId").loc[topid]

In [4]:
ratings = ratings[ratings["movieId"].isin(top.index)]

In [5]:
design = ratings.pivot(index="userId", columns="movieId", values="rating").fillna(2.5)
design_centered = design-design.mean()

In [6]:
X = design_centered.to_numpy()
U,M,V = np.linalg.svd(X, full_matrices=False)
V1 = V[:40].T

In [7]:
def recommendations(movie, data):
    movieid = top[top["title"]==movie].index[0]
    movieidx = np.argwhere(topid==movieid)[0]
    movie_to_compare = data[movieidx]
    
    distance = np.linalg.norm(data - movie_to_compare, axis=1)
    similar_ids = topid[distance.argsort()[:10]]
    similar_movies = top.loc[similar_ids]
    return similar_movies["title"].to_numpy()

In [8]:
print(f"Movies similar to:")
print(f"Toy Story: {recommendations('Toy Story (1995)', V1)}")
print(f"Iron Man: {recommendations('Iron Man (2008)', V1)}")

Movies similar to:
Toy Story: ['Toy Story (1995)' "There's Something About Mary (1998)"
 'GoldenEye (1995)' 'Aliens (1986)'
 'Léon: The Professional (a.k.a. The Professional) (Léon) (1994)'
 'Abyss, The (1989)' 'Grumpier Old Men (1995)' 'Crimson Tide (1995)'
 'Incredibles, The (2004)' 'Fugitive, The (1993)']
Iron Man: ['Iron Man (2008)' 'Breakfast Club, The (1985)' 'WALL·E (2008)'
 'Net, The (1995)' "One Flew Over the Cuckoo's Nest (1975)"
 'Star Wars: Episode V - The Empire Strikes Back (1980)'
 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)'
 'Godfather: Part II, The (1974)' 'Babe (1995)' 'Crimson Tide (1995)']
